In [1]:
# %%
import numpy as np
from scipy.spatial import Delaunay, minkowski_distance
import meshio
import pygmsh
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from mesh import Mesh
import scipy.sparse.linalg as spla
from icosphere import icosphere
from geometry import project_vector_to_plane

np.random.seed(42)

#with pygmsh.geo.Geometry() as geom:
#    geom.add_rectangle(0.0, 1.0, 0.0, 1.0, 0.0, mesh_size=0.01)
#    mesh = geom.generate_mesh()

#points = mesh.points
#simplices = mesh.cells_dict['triangle']
#point_normals = np.array([[0,0,1]]*len(simplices))
#mesh = Mesh(points, simplices, point_normals)

points, simplices = icosphere(7)
point_normals = points / np.linalg.norm(points,axis=-1,keepdims=True)
print(len(simplices))
mesh = Mesh(points, simplices, point_normals)

print(mesh.print_member_shapes())

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
980
barycenters = (3, 980)
barynormals = (3, 980)
areas = (980,)
owners = (1470,)
neighbours = (1470,)
edges_to_vertices = (1470, 2)
edge_lengths = (1470,)
edge_centers = (3, 1470)
edge_tangents = (3, 1470)
edge_bitangents = (3, 1470)
edge_normals = (3, 1470)
cells_to_edges = 980
edge_weighing_factor = (1470,)
skewness = (1470,)
skewness_vector = (3, 1470)
points = (492, 3)
point_normals = (492, 3)
simplices = (980, 3)
cf = (3, 1470)
d_cf = (1470,)
e = (3, 1470)
ef = (1470,)
owner_indices = (147

In [ ]:
# %% Init
# \phi = x^2 + y^2 + (xy)^2
xc = mesh.barycenters[:,0]
yc = mesh.barycenters[:,1]
zc = mesh.barycenters[:,2]

phi = 1.0/np.cosh(100*(xc - 0.5))/np.cosh(100*(yc - 0.5))

xf = mesh.edge_centers[:,0]
yf = mesh.edge_centers[:,1]
zf = mesh.edge_centers[:,2]

u = np.random.randn(*(len(xc),3))

for i in range(len(xc)):
    u[i,:] = project_vector_to_plane(u[i,:],mesh.barynormals[i,:])
outness = np.einsum("ij,ij->i",u, mesh.barynormals)
print("Initial outness: ", np.max(outness), np.min(outness))

# Find face flux
energy_profile = []
div_profile = []
for _ in range(50):
    uf = mesh.interpolate_field_cell_to_face(u)
    un = np.einsum("ij,ij->i", uf, mesh.edge_normals)
    un, du = mesh.helmholtz_projection(un)
    u += du

    e = mesh.get_total_kinetic_emergy(u)
    print("U: ", e, np.max(u), np.min(u))
    energy_profile.append(e)

    uf1 = mesh.interpolate_field_cell_to_face(u)
    un1 = np.einsum("ij,ij->i",uf1, mesh.edge_normals)
    div, max_div, min_div = mesh.divergence(un1)
    div_profile.append(div)
    print("Div: ", div, max_div, min_div)

    div, max_div, min_div = mesh.divergence(un)
    div_profile.append(div)
    print("Div corr: ", div, max_div, min_div)

    outness = np.einsum("ij,ij->i",u, mesh.barynormals)
    print("Outness: ", np.max(outness), np.min(outness))

In [ ]:
mesh = meshio.Mesh(
    points,
    [("triangle", simplices),],
    # Optionally provide extra data on points, cells, etc.
    #point_data={"T": [0.3, -1.2, 0.5, 0.7, 0.0, -3.0]},
    # Each item in cell data must match the cells array
    cell_data={"u": [u], "normals": [mesh.barynormals]},
)
mesh.write(
    "test_divergence.vtk",  # str, os.PathLike, or buffer/open file
    # file_format="vtk",  # optional if first argument is a path; inferred from extension
)

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(121)
ax1.plot(energy_profile)
ax2 = fig.add_subplot(122)
ax2.plot(div_profile)
plt.show()